# 124Te + double electron hole , #2 version

This notebook is meant to calculate background rejection and signal acceptance in a set of simulations (generated with Geant4 and later on clustered) that are within the energy region of interest (ROI) of the decay of 124Te* from its excited state of energy 2790.41 keV to the ground state. Two values are needed for the background rejection and signal acceptance to be calculated:

i) how many events within a set of simulations are within the ROI of this decay and

ii) how many events within a set of simulations correspond to 1 of 5 decay channels of this state of 124Te*. In reality, there are 9 possible decay channels for this state; however, the 4 that are not featured in this notebook amount only to 0.33% of the decays, and can therefore be ignored.

Each event under study contains one or more clusters. Firstly, the energies of each of these clusters, E_cluster, are Gaussian smeared with a sigma_S2, since the energy values obtained from Geant4 are exact and it is unlikely that they can be seen with such precision in a detector. On the other hand, the energies of each event are also smeared, but with a sigma_(S1+S2), since they can be read in the detector with more precision (S1+S2 analysis).

Then, the notebook checks whether or not the sum of all the E_cluster of one event, E_event, is within the ROI of the decay of 124Te* from its excited state of energy 2790.41 keV to the ground state. The ROI corresponds to

ROI = [E_true - 3sigma_S1+S2; E_true + 3sigma_S1+S2],

(in this case, E_true = 2790.41 keV). If E_event is not within the interval, the event is considered to be background. If it is within the interval, the event is saved as being within the ROI (calculation of i).
Then, the notebook makes all the possible combinations of the E_cluster of the event, and the sums of the energies of each combination, E_comb, are compared to the theoretical gamma energies corresponding to each decay channel, E_theo. Please note that E_comb can also correspond to one E_cluster (one possible combination of cluster energies is simply one cluster energy). If E_comb matches an E_theo of a certain channel, within a given absolute tolerance abs_tol, the notebook then combines and compares the remaining E_cluster of the event with the remaining E_theo of that decay channel, until a full correspondence is found. When this happens, the event is considered to fit in that channel.

If one event fits into several decay channels, two things can happen: either it is possible to define a hierarchy between them, and one decay channel can be selected in detriment of the other, or the two decay channels are equally probable, and the decay is classified as having taken place in an undetermined channel.

The total number of events that fit within the decay channels is then saved (calculation of ii).

It is important to highlight that, even though it is possible for the notebook to recognize two or more distinct clusters as fractions of the same energy deposition (meaning that a theoretical gamma energy can be "divided" between clusters), the opposite is not true - it is not possible for the notebook to recognize an event as a decay of 124Te* if two or more gamma energies are grouped within the same cluster. It therefore seems that using smaller cluster size while clustering the data can be beneficial.

Please note that the background rejection and the signal acceptance are not meant to be calculated for the same set of simulations. The background rejection should be calculated for isotopes other than 124Te* (potential background sources), and the signal acceptance only for 124Te* decays.

The following parameters have to be set:
1. Cluster size - this parameter is set before the data is clustered (so not on this notebook). Therefore, altering it in the input variables will not change the results. It is meant only to be used as a reference value. It is usually taken to be within the interval of 5 mm to 20 mm.
2. Minimum and maximum number of clusters, min_clusters and max_clusters - these are currently set to 2 and 11, respectively. Events with only 1 cluster cannot be fit into any decay channel in the notebook (any decay channel involves at least two energy depositions), and events with more than 11 clusters severely slow down the analysis. If the maximum number of clusters is set to 11, less than 1% of the events are lost.
3. Sigma used for smearing, sigma_E - this parameter corresponds to the percentage of Gaussian smearing of each cluster energy, and it usually ranges from 1% to 15% of its energy value. It is also used to set the absolute tolerance accepted between E_theo and E_comb.
4. Absolute tolerance between theoretical and simulation energies abs_tol - this corresponds to the maximum difference between the theoretical and simulated energy values, E_theo and E_comb, so that they are considered identical. It corresponds to nr_sigmas_abs_tol * sigma_E. The parameter sigma_E, as mentioned before, is also used for the Gaussian smearing of the E_cluster values. The parameter nr_sigmas_abs_tol corresponds to the number of sigmas taken as the absolute tolerance and is currently set to 3.
5. File that will be analyzed - this can either correspond to the decay chain of 124Te* or to decay chains of background sources. The Geant4 files that have been created and clustered and that are available for analysis are usually named in the following way: output_"material"_"isotope"_"i"_Proc.root (i corresponds to the run number). e.g. output_CryoMat_Tl208_17_Proc.root.
6. Main energy value, E_true - it is currently set as 2790.41 keV, as mentioned above. It might be necessary to change it if one wants to take into account the energy of the double-electron hole left in the atomic shell (64.457 keV). For now, it is not advisable to change it.
7. Number of sigmas used in ROI, nr_sigmas_ROI - even though the sigma_ROI value is fixed, the minimum and maximum energy values allowed in the ROI can be altered by changing the number of sigmas added and subtracted to it. It is currently set as 3 and it is not advisable to change it.

In [1204]:
#imports

import numpy as np
import ROOT
import root_numpy
import math
import itertools
from random import gauss
import uproot
import time
from scipy.stats import chi2
from statistics import mean

In [1205]:
#import energies via uproot

def get_data(filename):
    file = uproot.open(filename)
    tree = file["events"]
    energies = tree["Etot"].array()
    return energies

In [1206]:
#import positions via uproot

def get_xyz(filename):
    file = uproot.open(filename)
    tree = file["events"]
    x = tree["Xp"].array()
    y = tree["Yp"].array()
    z = tree["Zp"].array()
    pos = [x,y,z]
    return pos

In [1232]:
#input variables

cluster_size = 15
min_clusters = 2
max_clusters = 11
sigma_S2 = 0.05
sigma_abs_tol = sigma_S2
#sigma_abs_tol = 0.05
nr_sigmas_abs_tol = 3
material_isotope = "electrons"
E_true = 2790.41
nr_sigmas_ROI = 3

#fiducial volume
rmax = 1260
zmax = 1079.51
zshift = 50
n = 11

In [1233]:
cd /userdata/jd1076/MCProcessor/proc/ER_0vECEC

/userdata/jd1076/MCProcessor/proc/ER_0vECEC


In [1234]:
#cd /userdata/msilva/MCProcessor/proc

In [1240]:
#specify file to be analyzed (material, isotope)

def file_specification(which_simulation):
    
    energies = []
    positions = [[],[],[]]
    positions_simple = []
    
    if which_simulation.find("CryoMat_Bi214") != -1:    
        for i in range(1, 101):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_CryoMat_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
    
    if which_simulation.find("CryoMat_Tl208") != -1: 
        for i in range(1, 101):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_CryoMat_Tl208_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
        
    if which_simulation.find("CryoMat_Sc44") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_CryoMat_Sc44_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
        
    if which_simulation.find("Cu_Bi214") != -1:
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_Cu_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
               
    if which_simulation.find("Cu_Tl208") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_Cu_Tl208_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])

    if which_simulation.find("LXe_Bi214") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_LXe_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
               
    if which_simulation.find("PMT_Bi214") != -1: 
        for i in range(1, 1001):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_PMT_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
     
    if which_simulation.find("PMT_Tl208") != -1: 
        for i in range(1, 1001):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_PMT_Tl208_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
        
    if which_simulation.find("PTFE_Bi214") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_PTFE_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
        
    if which_simulation.find("PTFE_Tl208") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_PTFE_Tl208_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
                      
    if which_simulation.find("Ti_Bi214") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_Ti_Bi214_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
         
    if which_simulation.find("Ti_Tl208") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_Ti_Tl208_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
        
    if which_simulation.find("Ti_Sc44") != -1: 
        for i in range(1, 11):
            filename = f"/userdata/jd1076/MCProcessor/proc/ER_0vECEC/output_Ti_Sc44_{i}_Proc.root"
            energy = get_data(filename)
            position = get_xyz(filename)
            for j in range(len(energy)):
                energies.append(energy[j][:])    
            for k in range(3):
                for l in range(len(position[k])):
                    positions[k].append(position[k][l])
                
    if which_simulation.find("Te124_15mm") != -1:
        filename = '/userdata/msilva/MCProcessor/proc/Te124_15mm_Proc.root'
        energy = get_data(filename)
        position = get_xyz(filename)
        for j in range(len(energy)):
            energies.append(energy[j][:])    
        for i in range (len(position)):
            positions_simple.append(position[i][:])  
            
    if which_simulation.find("electrons") != -1:
        filename = '/userdata/msilva/MCProcessor/proc/electrons_Proc.root'
        energy = get_data(filename)
        position = get_xyz(filename)
        for j in range(len(energy)):
            energies.append(energy[j][:])    
        for i in range (len(position)):
            positions_simple.append(position[i][:])
            
            
    if which_simulation.find("gammas") != -1:
        filename = '/userdata/msilva/MCProcessor/proc/gammas_Proc.root'
        energy = get_data(filename)
        position = get_xyz(filename)
        for j in range(len(energy)):
            energies.append(energy[j][:])    
        for i in range (len(position)):
            positions_simple.append(position[i][:])     
            
    #return energies, positions
    return energies, positions_simple

In [1241]:
#make list of energies

energies, positions = file_specification(material_isotope)

In [1243]:
#function that smears event energies (sigma S1+S2)

def smearing_event(energies):
    event_energies = []
    a = 0.3171
    b = 0.0015
    for i in range(len(energies)):
        total_energy = np.sum(energies[i])
        sigma = total_energy * (a / math.sqrt(total_energy) + b)
        E_new = gauss(total_energy, sigma)
        event_energies.append(E_new)
    return event_energies

In [1244]:
#function that smears cluster energies (sigma S2)

def smearing_cluster(energies, sigma_S2):
    for i in range(len(energies)):
        for j in range(len(energies[i])):
            sigma = energies[i][j] * sigma_S2
            E_new = gauss(energies[i][j], sigma)
            energies[i][j] = E_new
    return energies

In [1245]:
#define absolute tolerance function (sigma S2)

def absolute_tolerance(energies, sigma_S2, nr_sigmas_abs_tol):
    sigma = energies * sigma_S2
    tol = nr_sigmas_abs_tol * sigma
    return tol

In [1246]:
#define ROI (sigma S1+S2)

def region_of_interest(main_energy, nr_sigmas_ROI):
    a = 0.3171
    b = 0.0015
    En_range = []
    sigma = main_energy * (a / math.sqrt(main_energy) + b)
    E_new1 = main_energy - nr_sigmas_ROI*sigma
    E_new2 = main_energy + nr_sigmas_ROI*sigma
    En_range.append(E_new1)
    En_range.append(E_new2)
    return En_range

En_range = region_of_interest(E_true, nr_sigmas_ROI)

In [1247]:
#define error margins if Poisson distribution is used

def poisson_interval(value, a = 0.318): #value is in 68% CI
    bounds = []
    interval = []
    minimum_interval, maximum_interval = (chi2.ppf(a/2, 2*value) / 2, chi2.ppf(1-a/2, 2*value + 2) / 2)
    if value == 0: 
        minimum_interval = 0.0
    minus = value - minimum_interval
    plus = maximum_interval - value
    bounds.append(minus)
    bounds.append(plus)
    interval.append(minimum_interval)
    interval.append(maximum_interval)
    return bounds

In [1248]:
#define error margins to be used in final results (Poisson or Gaussian)

def error_margin(number):
    
    if number >= 20:
        error = math.sqrt(number)
    else:
        inlist = poisson_interval(number, 0.318)
        maximum = max(inlist)
        error = maximum  
        
    return error

In [1249]:
#cut on fiducial volume

def fiducial_volume(positions, event, rmax, zmax, zshift, n):
    
    clusters_within_FV = []
    
    for i in range(len(positions[0][event])):
        x = positions[0][event][i]
        y = positions[1][event][i]
        z = positions[2][event][i]
        r = math.sqrt(x**2+y**2)
        
        if ((r/rmax)**n+((z+zshift)/zmax)**n) <= 1:
            clusters_within_FV.append(True)
        else:
            clusters_within_FV.append(False)
            
    if all(clusters_within_FV):        
        return True
    
    else:        
        return False

In [1250]:
#theoretical values of gamma energies of each decay channel

E_comb_A = [64.457, 602.7260, 2188]
E_comb_B = [64.457, 602.7260, 722.782, 1464.66]
E_comb_C = [64.457, 1325.504, 1464.66]
E_comb_D = [64.457, 602.7260, 751.5, 1436.689]
E_comb_E = [64.457, 751.5, 2039.36]

E_comb = [E_comb_A, E_comb_B, E_comb_C, E_comb_D, E_comb_E]

level_list = []

for i in range(len(E_comb)):
    nr_levels = len(E_comb[i])
    level_list.append(nr_levels)

In [1251]:
#smearing event energies

energies_events = smearing_event(energies)

In [1252]:
#smearing cluster energies

energies_clusters = smearing_cluster(energies, sigma_S2)

In [1253]:
#E-test

level = 0
counter_not_clusters = 0
counter_not_ROI = 0
counter_not_FV = 0
counter_ROI = 0
D = {}

for event in range(len(energies_events)):
    D[event] = [False, False, False, False, False]
    
    if (len(energies_clusters[event]) >= min_clusters) and (len(energies_clusters[event]) <= max_clusters):
        if En_range[0] <= energies_events[event] <= En_range[1]:
            if fiducial_volume(positions, event, rmax, zmax, zshift, n):
                counter_ROI += 1
                level = 1
                for cluster_0 in range(1, len(energies_clusters[event])):
                    combs_0 = list(itertools.combinations(energies_clusters[event], cluster_0))
                    for combination_0 in combs_0:
                        for channel in range(5):
                            if D[event][channel] == False:
                                if math.isclose(sum(combination_0), E_comb[channel][0], abs_tol = absolute_tolerance(E_comb[channel][0], sigma_abs_tol, nr_sigmas_abs_tol)):
                                    level = 2
                                    remaining_energies_1 = set(combination_0) ^ set(energies_clusters[event])
                                    for cluster_1 in range(1, len(remaining_energies_1)):
                                        combs_1 = list(itertools.combinations(remaining_energies_1, cluster_1))
                                        for combination_1 in combs_1:
                                            if math.isclose(sum(combination_1),E_comb[channel][1], abs_tol = absolute_tolerance(E_comb[channel][1], sigma_abs_tol, nr_sigmas_abs_tol)):
                                                level = 3
                                                if level_list[channel] == level:
                                                    leftover_energy = sum(energies_clusters[event]) - sum(combination_0) - sum(combination_1)
                                                    if math.isclose(leftover_energy, E_comb[channel][2], abs_tol = absolute_tolerance(E_comb[channel][2], sigma_abs_tol, nr_sigmas_abs_tol)):
                                                        D[event][channel] = True
                                                        
                                                else:
                                                    remaining_energies_2 = set(combination_0) ^ set(combination_1) ^ set(energies_clusters[event])
                                                    for cluster_2 in range(1, len(remaining_energies_2)):
                                                        combs_2 = list(itertools.combinations(remaining_energies_2, cluster_2))
                                                        for combination_2 in combs_2:
                                                            if math.isclose(sum(combination_2),E_comb[channel][2], abs_tol = absolute_tolerance(E_comb[channel][1], sigma_abs_tol, nr_sigmas_abs_tol)):
                                                                leftover_energy = sum(energies_clusters[event]) - sum(combination_0) - sum(combination_1) - sum(combination_2)
                                                                if math.isclose(leftover_energy, E_comb[channel][3], abs_tol = absolute_tolerance(E_comb[channel][2], sigma_abs_tol, nr_sigmas_abs_tol)):
                                                                    D[event][channel] = True
            else:
                counter_not_FV += 1
                                
        else:
            counter_not_ROI += 1 
        
    else:
        counter_not_clusters += 1                     
                        

In [1255]:
#assignment

counter_channels = 0
counter_background = 0

for event in range(len(D)):
    
    if En_range[0] <= energies_events[event] <= En_range[1] and (len(energies_clusters[event]) >= min_clusters) and (len(energies_clusters[event]) <= max_clusters) and fiducial_volume(positions, event, rmax, zmax, zshift, n):
        
        if D[event][0] == False and D[event][1] == False and D[event][2] == False and D[event][3] == False and D[event][4] == False:
            counter_background += 1.0
            
        else:
            counter_channels += 1.0

In [1256]:
#generating error margins for the results

error_channels = error_margin(counter_channels)
error_background = error_margin(counter_background)
error_ROI = error_margin(counter_ROI)

In [1257]:
#printing info

print(f"""
Events in channels: {counter_channels} ± {error_channels}
Background events: {counter_background} ± {error_background}
Events in ROI: {counter_ROI} ± {error_ROI}
""")


Events in channels: 5036.0 ± 70.96477999684069
Background events: 7875.0 ± 88.74119674649424
Events in ROI: 12911 ± 113.62658139713612



In [1258]:
#calculating signal acceptance

alpha = counter_channels/counter_ROI * 100
delta_alpha = (alpha * math.sqrt((error_channels/counter_channels)**2 + (error_ROI/counter_ROI)**2))

print(f"""
Material and isotope under study: {material_isotope}
Cluster size: {cluster_size} mm
Minimum number of clusters per event: {min_clusters}
Maximum number of clusters per event: {max_clusters}
Value of sigma_S2 {sigma_S2*100} %
--------------------------------------------------------------------------
The signal acceptance alpha is ({alpha} ± {delta_alpha})%
""")


Material and isotope under study: electrons
Cluster size: 15 mm
Minimum number of clusters per event: 2
Maximum number of clusters per event: 11
Value of sigma_S2 5.0 %
--------------------------------------------------------------------------
The signal acceptance alpha is (39.00549918673999 ± 0.6480357579198662)%



In [1259]:
#calculating background rejection

epsilon = (counter_background/counter_ROI) * 100
delta_epsilon = (epsilon * math.sqrt(error_background/counter_background) **2 + (error_ROI/counter_ROI) **2)

print(f"""
Material and isotope under study: {material_isotope}
Cluster size: {cluster_size} mm
Minimum number of clusters per event: {min_clusters}
Maximum number of clusters per event: {max_clusters}
Value of sigma_S2: {sigma_S2*100} %
--------------------------------------------------------------------------
The background rejection epsilon is ({epsilon} ± {delta_epsilon})%
""")


Material and isotope under study: electrons
Cluster size: 15 mm
Minimum number of clusters per event: 2
Maximum number of clusters per event: 11
Value of sigma_S2: 5.0 %
--------------------------------------------------------------------------
The background rejection epsilon is (60.99450081326001 ± 0.6874076116992816)%



In [1260]:
print(f"""
If the notebook is correct, the result of alpha + epsilon should be 100%.
It is {alpha + epsilon}%.
""")


If the notebook is correct, the result of alpha + epsilon should be 100%.
It is 100.0%.

